<a href="https://colab.research.google.com/github/magnus-rex1/cattle-behavior-classifier/blob/main/classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install mlflow
!pip install pyngrok

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.7/19.7 MB 37.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 195.4/195.4 kB 14.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.0/53.0 kB 4.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.4/233.4 kB 18.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 147.6/147.6 kB 10.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 128.2/128.2 kB 12.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 80.2/80.2 kB 5.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.7/78.7 kB 2.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 6.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 202.9/202.9 kB 17.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.8/52.8 kB 5.5 MB/s eta 0:00:00
  Attempting uninstall: packaging
    Found existing installation: packaging 24.0
    Uninstalling packa

In [15]:
#Include all the necessary libraries
import pandas as pd
import numpy as np

from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score, accuracy_score
from sklearn.preprocessing import StandardScaler, MinMaxScaler

import mlflow
import mlflow.sklearn
from mlflow.models import infer_signature
import joblib
import subprocess
from pyngrok import ngrok, conf
import getpass
from datetime import datetime
# from mlflow.models import infer_signature
# import joblib

**MLFlow UI backend**

In [3]:
MLFLOW_TRACKING_URI = "sqlite:///mlflow.db"
subprocess.Popen(["mlflow", "ui", "--backend-store-uri", MLFLOW_TRACKING_URI])

<Popen: returncode: None args: ['mlflow', 'ui', '--backend-store-uri', 'sqli...>

# MLFlow experiment name

In [5]:
mlflow.set_tracking_uri(MLFLOW_TRACKING_URI)
# mlflow will create an experiment if it doesn't exist
experiment_name = "behavior-classification-experiment"
run_name = datetime.now().strftime("%Y%m%d_%H%M%S")

In [6]:
print("Enter your authtoken, which can be copied from https://dashboard.ngrok.com/auth")
conf.get_default().auth_token = getpass.getpass()
port=5000
public_url = ngrok.connect(port).public_url
print(f' * ngrok tunnel \"{public_url}\" -> \"http://127.0.0.1:{port}\"')

Enter your authtoken, which can be copied from https://dashboard.ngrok.com/auth
··········
 * ngrok tunnel "https://b2d5-35-184-154-39.ngrok-free.app" -> "http://127.0.0.1:5000"


# Mount google drive

In [16]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [9]:
# Load dataset from the CSV file
cow1 = pd.read_csv('/content/drive/MyDrive/JapaneseBeef/cow1.csv')
cow2 = pd.read_csv('/content/drive/MyDrive/JapaneseBeef/cow2.csv')
cow3 = pd.read_csv('/content/drive/MyDrive/JapaneseBeef/cow3.csv')
cow4 = pd.read_csv('/content/drive/MyDrive/JapaneseBeef/cow4.csv')

,TimeStamp_UNIX,TimeStamp_JST,AccX,AccY,AccZ,Label
0,1591928577026,11:22:57.026000,0.010,0.025,-0.991,NaN
1,1591928577066,11:22:57.066000,0.010,0.025,-0.991,NaN
2,1591928577106,11:22:57.106000,0.010,0.024,-0.991,NaN
3,1591928577146,11:22:57.146000,0.010,0.025,-0.991,NaN
4,1591928577186,11:22:57.186000,0.010,0.025,-0.991,NaN
...,...,...,...,...,...,...
311871,1591950107816,17:21:47.816000,-0.225,-0.049,0.839,ETC
311872,1591950107856,17:21:47.856000,-0.117,-0.116,0.831,ETC
311873,1591950107896,17:21:47.896000,-0.085,0.143,0.745,NaN
311874,1591950107936,17:21:47.936000,-0.229,-0.182,0.987,NaN


In [ ]:
cow1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 311876 entries, 0 to 311875
Data columns (total 6 columns):
 #   Column          Non-Null Count   Dtype 
---  ------          --------------   ----- 
 0   TimeStamp_UNIX  311876 non-null  object
 1   TimeStamp_JST   311876 non-null  object
 2   AccX            311876 non-null  object
 3   AccY            311876 non-null  object
 4   AccZ            311876 non-null  object
 5   Label           160627 non-null  object
dtypes: object(6)
memory usage: 14.3+ MB


In [ ]:
# cow1[['TimeStamp_UNIX', 'AccX', 'AccY', 'AccZ']] = cow1[['TimeStamp_UNIX', 'AccX', 'AccY', 'AccZ']].apply(pd.to_numeric)

In [10]:
# converting from mg to m/s^2
def convert_acc_units(df):
    df['AccX'] = (df['AccX']/1000)*9.81
    df['AccY'] = (df['AccY']/1000)*9.81
    df['AccZ'] = (df['AccZ']/1000)*9.81
    df['AccMag'] = np.sqrt(df['AccX']**2 + df['AccY']**2 + df['AccZ']**2)

    return df

cow1 = convert_acc_units(cow1)

behaviors = {
    'RES': 1,  # Resting in standing position
    'RUS': 2,  # Ruminating in standing position
    'MOV': 3,  # Moving
    'GRZ': 4,  # Grazing
    'SLT': 5,  # Salt licking
    'FES': 6,  # Feeding in stanchion
    'DRN': 7,  # Drinking
    'LCK': 8,  # Licking
    'REL': 9,  # Resting in lying position
    'URI': 10,  # Urinating
    'ATT': 11, # Attacking
    'ESC': 12, # Escaping
    'BMN': 13, # Being mounted
    'ETC': 14, # Other behaviors
    'BLN': 15  # Data without video, no label
}

In [11]:
# encode 'Label column
def encode_label_column(df):
  df['behavior'] = df['Label'].map(behaviors)
  # preserve the nans in the original column
  df['behavior'] = df['behavior'].where(~df['Label'].isnull(), pd.NA)

  return df

cow1 = encode_label_column(cow1)


from sklearn.impute import SimpleImputer

def simple_impute(df):
    imp_mean = SimpleImputer(missing_values=np.nan, strategy='mean')
    df['behavior'] = imp_mean.fit_transform(df[['behavior']])
    df['behavior'] = df['behavior'].apply(lambda x: round(x, 1))
    return df

cow1 = simple_impute(cow1)

In [14]:
mlflow.set_experiment(experiment_name=experiment_name)
experiment = mlflow.get_experiment_by_name(experiment_name)

with mlflow.start_run(experiment_id=experiment.experiment_id):
    features = ['AccX', 'AccY', 'AccZ', 'AccMag']
    target_variable = 'behavior'

    X = cow1[features]
    y = cow1[target_variable]

    y = y.astype('int')

    # standardize the dataset
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X)

    # split the data into training and testing sets
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

    # define and train the random forest model
    model = RandomForestClassifier(n_estimators=100, random_state=0)
    model.fit(X_train, y_train)

    # make predictions on training data
    y_pred = model.predict(X_test)

    # evaluate accuracy
    accuracy = accuracy_score(y_test, y_pred)
    mse = mean_squared_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)

    # print evaluation metrics
    print(f'Accuracy: {accuracy}')
    print(f'Mean Squared Error: {mse}')
    print(f'R-squared: {r2}')

    # Log metrics to MLflow
    mlflow.log_param("features", features)
    mlflow.log_param("target_variable", target_variable)
    mlflow.log_metric("mse", mse)
    mlflow.log_metric("r2", r2)

    # Log model to MLflow
    # mlflow.sklearn.log_model(model, "model")
     # model signature
    signature = infer_signature(X_train, y_pred)
    model_info = mlflow.sklearn.log_model(sk_model=model, artifact_path="model", signature=signature)
    sklearn_pyfunc = mlflow.pyfunc.load_model(model_uri=model_info.model_uri)

    # Save the model using joblib
    joblib.dump(model, 'behavior_model.pkl')


    # Save the model path for future reference
    model_path = "behavior_model.pkl"

    # Log the model path to MLflow
    mlflow.log_artifact(model_path)

# end run
mlflow.end_run()


Accuracy: 0.7232589457483648
Mean Squared Error: 14.659147749134283
R-squared: 0.12530092549370497


2024/03/18 09:53:43 WARNING mlflow.utils.environment: Encountered an unexpected error while inferring pip requirements (model URI: /tmp/tmpo1vapqw3/model/model.pkl, flavor: sklearn). Fall back to return ['scikit-learn==1.2.2', 'cloudpickle==2.2.1']. Set logging level to DEBUG to see the full traceback. 
/usr/local/lib/python3.10/dist-packages/_distutils_hack/__init__.py:33: UserWarning: Setuptools is replacing distutils.
  warnings.warn("Setuptools is replacing distutils.")
